In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import Sampler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensor

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings(action='ignore')

N_EPOCHS = 1000
BATCH_SIZE = 128
LEARNING_RATE = 0.0005
PAITIENCE = 30

IM_HEIGHT = 256
IM_WIDTH = 256
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
train_pos, train_neg  = 'data/tumor_classification/train/positive', 'data/tumor_classification/train/negative'
valid_pos, valid_neg  = 'data/tumor_classification/valid/positive', 'data/tumor_classification/valid/negative'
test_pos, test_neg    = 'data/tumor_classification/test/positive', 'data/tumor_classification/test/negative'

def generate_dataframe(path_positive, path_negative):
    df = pd.concat([
            pd.DataFrame({"image": glob.glob(path_positive + "/*.png"), "target": 1}),
            pd.DataFrame({"image": glob.glob(path_negative + "/*.png"), "target": 0}),
        ])
    
    return df

train_df = generate_dataframe(train_pos, train_neg)
valid_df = generate_dataframe(valid_pos, valid_neg)
test_df  = generate_dataframe(test_pos, test_neg)

In [3]:
train_transforms = A.Compose([
    A.augmentations.Resize(width=IM_HEIGHT, height=IM_WIDTH, p=1.0),
    A.augmentations.HorizontalFlip(p=0.5),
    A.augmentations.VerticalFlip(p=0.5),
    A.augmentations.ShiftScaleRotate(p=0.5, shift_limit=0.05, scale_limit=0.1, rotate_limit=45),
    A.ElasticTransform(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RandomBrightness(p=0.5),
    A.Transpose(p=0.5),
    A.GaussNoise(p=0.3),
    A.GaussianBlur(p=0.3),
    # A.ImageCompression(p=0.3),
    A.augmentations.Normalize(p=1.0),
    ToTensor(),
])

valid_transforms = A.Compose([
    A.augmentations.Resize(width=IM_HEIGHT, height=IM_WIDTH, p=1.0),
    A.augmentations.Normalize(p=1.0),
    ToTensor(),
])


class TumorDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image  = cv2.imread(self.df.iloc[idx, 0])
        target = self.df.iloc[idx, 1]

        augmented = self.transforms(image=image)
        image = augmented['image']  
        
        return image, target
    
    def get_labels(self):
        return list(self.df.target.values)

    
class BalanceClassSampler(Sampler):
    """Abstraction over data sampler.
    Allows you to create stratified sample on unbalanced classes.
    """

    def __init__(self, labels, mode="downsampling"):
        """
        Args:
            labels (List[int]): list of class label
                for each elem in the datasety
            mode (str): Strategy to balance classes.
                Must be one of [downsampling, upsampling]
        """
        super().__init__(labels)

        labels = np.array(labels)
        samples_per_class = {
            label: (labels == label).sum() for label in set(labels)
        }

        self.lbl2idx = {
            label: np.arange(len(labels))[labels == label].tolist()
            for label in set(labels)
        }

        if isinstance(mode, str):
            assert mode in ["downsampling", "upsampling"]

        if isinstance(mode, int) or mode == "upsampling":
            samples_per_class = (
                mode
                if isinstance(mode, int)
                else max(samples_per_class.values())
            )
        else:
            samples_per_class = min(samples_per_class.values())

        self.labels = labels
        self.samples_per_class = samples_per_class
        self.length = self.samples_per_class * len(set(labels))

    def __iter__(self):
        """
        Yields:
            indices of stratified sample
        """
        indices = []
        for key in sorted(self.lbl2idx):
            replace_ = self.samples_per_class > len(self.lbl2idx[key])
            indices += np.random.choice(
                self.lbl2idx[key], self.samples_per_class, replace=replace_
            ).tolist()
        assert len(indices) == self.length
        np.random.shuffle(indices)

        return iter(indices)

    def __len__(self) -> int:
        """
        Returns:
             length of result sample
        """
        return self.length
    
    
train_dataset = TumorDataset(df=train_df, transforms=train_transforms)
# train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False, 
    sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode='downsampling'))

valid_dataset = TumorDataset(df=valid_df, transforms=valid_transforms)
# valid_iterator = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
valid_iterator = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False,
    sampler=BalanceClassSampler(labels=valid_dataset.get_labels(), mode='downsampling'))

test_dataset = TumorDataset(df=test_df, transforms=valid_transforms)
test_iterator = DataLoader(test_dataset, batch_size=1, num_workers=4, shuffle=False)

In [4]:
device       = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model        = timm.create_model('nf_resnet50', num_classes=2, pretrained=True).to(device)
optimizer    = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler    = ReduceLROnPlateau(optimizer, 'min')
class_weight = torch.tensor([1.0, 1.0]).to(device)
criterion    = nn.CrossEntropyLoss(weight=class_weight)

for name, param in model.named_parameters():
    param.requires_grad = True
#     if name[:4] == "head":
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

In [5]:
def train(model, iterator, criterion, optimizer, device=device):  
    model.train()
    epoch_loss = 0
    correct    = 0    
    
    for image, target in iterator:
        optimizer.zero_grad()
        image  = image.to(device)
        target = target.long().to(device)
        
        output = model(image).squeeze()
        
        loss = criterion(output, target)
        loss.backward()
        epoch_loss += loss.item()
        
        pred     = torch.argmax(output, axis=1)
        correct += (pred == target).sum()

        optimizer.step()

    return epoch_loss / len(iterator.dataset), correct / len(iterator.dataset)


@torch.no_grad()
def evaluate(model, iterator, criterion, device=device):
    model.eval()
    epoch_loss = 0
    correct    = 0 
    
    for image, target in iterator:
        image  = image.to(device)
        target = target.long().to(device)

        output = model(image).squeeze()

        loss = criterion(output, target)
        epoch_loss += loss.item()

        pred     = torch.argmax(output, axis=1)
        correct += (pred == target).sum()
            
    return epoch_loss / len(iterator.dataset), correct / len(iterator.dataset)


@torch.no_grad()
def predict(model, iterator, device=device):
    model.eval()
    pred = []
    true = []
    
    for image, target in iterator:
        image  = image.to(device)
        target = target.long().to(device)

        output = model(image)

        pred.append(output.to("cpu").tolist()[0])
        true.append(target.to("cpu").tolist()[0])

    return np.argmax(pred, axis=1), true


def print_train_log(epoch_num, train_loss, valid_loss, train_acc, valid_acc):
    print(f"EPOCH: {epoch_num:04}")
    print(f"Train loss: {round(train_loss, 4)}\tTrain acc : {round(float(train_acc), 4)}\tValid loss: {round(valid_loss, 4)}\tValid acc : {round(float(valid_acc), 4)}")
    
    
def compute_test_metrics(true, pred):
    confusion_mat = confusion_matrix(true, pred)
    accuracy      = accuracy_score(true, pred)
    precision     = precision_score(true, pred)
    recall        = recall_score(true, pred)
    f1            = f1_score(true, pred)
    
    return confusion_mat, accuracy, precision, recall, f1


def print_test_log(accuracy, precision, recall, f1, epoch_num=None):
    if epoch_num:
        print(f"EPOCH: {epoch_num:04} prediction results ")
    else:
        print("prediction results")
        
    print(f"confusion matrix\n{confusion_mat}")
    print(f"accuracy score  : {round(accuracy, 4)}")
    print(f"precision score : {round(precision, 4)}")
    print(f"recall score    : {round(recall, 4)}")
    print(f"f1 score        : {round(f1, 4)}")

In [ ]:
start_epoch = 0
if len(glob.glob("output/tumor_classification/nfnet/*.txt")) != 0:
    print("load trained model ... ")
    start_epoch = len(glob.glob("output/tumor_classification/nfnet/*.txt")) - 1 
    model.load_state_dict(torch.load('weights/tumor_classification/best_nfnet.pt'))

n_paitience = 0
best_valid_loss = float('inf')

optimizer.zero_grad()
optimizer.step()

for epoch_num in range(start_epoch, N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    scheduler.step(valid_loss)
    
    print_train_log(epoch_num, train_loss, valid_loss, train_acc, valid_acc)
    with open("output/tumor_classification/nfnet/log.txt", "a") as f:
        f.write("epoch: {0:04d} train loss: {1:.4f}, valid loss: {2:.4f}, train Acc: {3:.4f}, valid Acc: {4:.4f}\n".format(epoch_num, train_loss, valid_loss, train_acc, valid_acc))

    if n_paitience < PAITIENCE:
        if best_valid_loss > valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'weights/tumor_classification/best_nfnet.pt')
            n_paitience = 0
        elif best_valid_loss <= valid_loss:
            n_paitience += 1
    else:
        print("Early stop!")
        model.load_state_dict(torch.load('weights/tumor_classification/best_nfnet.pt'))
        break
        
    if epoch_num % 1 == 0:
        
        pred, true = predict(model, test_iterator)
        confusion_mat, accuracy, precision, recall, f1 = compute_test_metrics(true, pred)
        
        print_test_log(accuracy, precision, recall, f1, epoch_num=epoch_num)
        with open("output/tumor_classification/nfnet/epoch_{0:04d}_eval_metrics.txt".format(epoch_num), "a") as f:
            f.write("accuracy score: {0:.4f}, precision score: {1:.4f}, recall score: {2:.4f}, f1 score: {3:.4f}\n".format(accuracy, precision, recall, f1))
         

load trained model ... 
EPOCH: 0026
Train loss: 0.0006	Train acc : 0.5909	Valid loss: 0.0005	Valid acc : 0.5968
EPOCH: 0026 prediction results 
confusion matrix
[[ 706   39]
 [  22 1631]]
accuracy score  : 0.9746
precision score : 0.9766
recall score    : 0.9867
f1 score        : 0.9816
EPOCH: 0027
Train loss: 0.0007	Train acc : 0.5861	Valid loss: 0.0005	Valid acc : 0.5945
EPOCH: 0027 prediction results 
confusion matrix
[[ 739    6]
 [ 121 1532]]
accuracy score  : 0.947
precision score : 0.9961
recall score    : 0.9268
f1 score        : 0.9602
EPOCH: 0028
Train loss: 0.0007	Train acc : 0.5878	Valid loss: 0.0007	Valid acc : 0.5861
EPOCH: 0028 prediction results 
confusion matrix
[[ 668   77]
 [  13 1640]]
accuracy score  : 0.9625
precision score : 0.9552
recall score    : 0.9921
f1 score        : 0.9733
EPOCH: 0029
Train loss: 0.0006	Train acc : 0.5915	Valid loss: 0.0004	Valid acc : 0.6001
EPOCH: 0029 prediction results 
confusion matrix
[[ 730   15]
 [  63 1590]]
accuracy score  : 0.9

EPOCH: 0057 prediction results 
confusion matrix
[[ 731   14]
 [  27 1626]]
accuracy score  : 0.9829
precision score : 0.9915
recall score    : 0.9837
f1 score        : 0.9875
EPOCH: 0058
Train loss: 0.0003	Train acc : 0.6088	Valid loss: 0.0003	Valid acc : 0.6093
EPOCH: 0058 prediction results 
confusion matrix
[[ 730   15]
 [  37 1616]]
accuracy score  : 0.9783
precision score : 0.9908
recall score    : 0.9776
f1 score        : 0.9842
EPOCH: 0059
Train loss: 0.0003	Train acc : 0.6082	Valid loss: 0.0002	Valid acc : 0.611
EPOCH: 0059 prediction results 
confusion matrix
[[ 733   12]
 [  27 1626]]
accuracy score  : 0.9837
precision score : 0.9927
recall score    : 0.9837
f1 score        : 0.9881
EPOCH: 0060
Train loss: 0.0003	Train acc : 0.6076	Valid loss: 0.0002	Valid acc : 0.6104
EPOCH: 0060 prediction results 
confusion matrix
[[ 731   14]
 [  27 1626]]
accuracy score  : 0.9829
precision score : 0.9915
recall score    : 0.9837
f1 score        : 0.9875
EPOCH: 0061
Train loss: 0.0003	Tr

In [10]:
model.load_state_dict(torch.load('weights/tumor_classification/best_nfnet.pt'))
pred, true = predict(model, test_iterator)
confusion_mat, accuracy, precision, recall, f1 = compute_test_metrics(true, pred)

print_test_log(accuracy, precision, recall, f1)

prediction results
confusion matrix
[[ 731   14]
 [  32 1621]]
accuracy score  : 0.9808
precision score : 0.9914
recall score    : 0.9806
f1 score        : 0.986
